In [ ]:
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, RGCNConv

from utils.path_manage import get_files

In [ ]:
triplets, lookup, ASD_dictionary, BCE_dictionary, Edge_list, Edge_features = get_files()
edge_index = torch.tensor(Edge_list, dtype=torch.long)
edge_features = torch.tensor(Edge_features, dtype=torch.float)
edge_type = torch.tensor([x.index(max(x)) for x in Edge_features], dtype=torch.long)

In [ ]:
print(edge_index[0:10])
print(triplets[0])

In [ ]:
import numpy as np
fake_edges = np.random.randint(low=0,high=10,size=(10,2))

In [ ]:
fake_index = torch.tensor(fake_edges, dtype=torch.long)

In [ ]:
data = Data(edge_index=edge_index[0:10].t().contiguous(), edge_attr=edge_features[0:10], edge_type=edge_type[0:10])
# data = Data(edge_index=fake_index.t().contiguous(), edge_attr=edge_features[0:10], edge_type=edge_type[0:10])

In [ ]:
# this code is designed for graph classification not link prediction. 

class Net(torch.nn.Module):
    def __init__(self, data, hidden_dim):
        super(Net, self).__init__()
        # Should add embedding layer at start  ----   self.emb_e = torch.nn.Embedding(data.num_nodes, hidden_dim, padding_idx=0)        
        self.emb_rel = torch.nn.Embedding(data.num_edge_features+1, hidden_dim, padding_idx=0)

        self.conv1 = RGCNConv(in_channels=data.num_nodes, out_channels=hidden_dim, num_relations=data.num_edge_features, num_bases=2)
        self.conv2 = RGCNConv(in_channels=hidden_dim, out_channels=hidden_dim, num_relations=data.num_edge_features, num_bases=2)
             
    def forward(self, edge_index, edge_type, rel):
        # Should add embedding layer to x? to do that you'll need to pass nodes to the forward, which is not done atm.  
        x = F.relu(self.conv1(None, edge_index, edge_type))
        x = self.conv2(x, edge_index, edge_type)
        rel_embedded = self.emb_rel(rel)

        return x, rel_embedded

    def distmult(self, out, triplets):
        obj = out[0][triplets[:,0]] 
        rel = out[1][range(len(triplets))]   # use rel = out[1][triplets[0:10,0]] or something similar when your code actually works
        sub = out[0][triplets[:,2]]
        distmult = torch.sum(sub * rel * obj, dim=1)  # print(obj, rel, sub)
        return distmult 
        

model = Net(data=data, hidden_dim = 5)
model.parameters

In [ ]:
out = model(data.edge_index, data.edge_type, data.edge_type)

In [ ]:
score = model.distmult(out, triplets[0:10])

In [ ]:
# you need the score over all tails for BCE!!!
print(score)

In [ ]:
obj = out[0][sample[:,0]]
# use rel = out[1][triplets[0:10,0]] or something similar when your code actually works
rel = out[1][range(len(sample))]
sub = out[0][sample[:,2]]
# print(obj, rel, sub)
distmult = torch.sum(obj * rel * sub, dim=1)
print(distmult)

In [ ]:
print(sample[:,2])

In [ ]:
out[0].transpose(0,1).shape

In [ ]:
all_scores = torch.mm((obj*sub),out[0].transpose(0,1))

In [ ]:
loss = torch.nn.CrossEntropyLoss()

output = loss(all_scores, torch.LongTensor(sample[:,2]))


In [ ]:
print(output)

In [ ]:
output.backward()

In [ ]:
print(model.emb_rel.weight.transpose(1, 0))


In [ ]:
data.edge_index

In [ ]:
pred = torch.mm(out[0], )

In [ ]:
triplets[0:2]

In [ ]:
distmult = model.distmult(out, triplets[0:1])
print(distmult)

In [ ]:
model

In [ ]:
for param in model.parameters():
    print(type(param.data), param.size())

In [ ]:
out = model(data.edge_index, data.edge_type)
# this is a call to the forward function!!!

In [ ]:
# okay so your model gives out node embeddings. Now to do a distmult on these bad boys.  

In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Entities
from torch_geometric.nn import RGCNConv

name = 'MUTAG'
path = osp.join(
    osp.dirname(osp.realpath('__file__')), '..', 'data', 'Entities', name)
dataset = Entities(path, name)
data = dataset[0]

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = RGCNConv(
            data.num_nodes, 16, dataset.num_relations, num_bases=30)
        self.conv2 = RGCNConv(
            16, dataset.num_classes, dataset.num_relations, num_bases=30)

    def forward(self, edge_index, edge_type, edge_norm):
        x = F.relu(self.conv1(None, edge_index, edge_type))
        x = self.conv2(x, edge_index, edge_type)
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
print(model.parameters)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)


def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.edge_index, data.edge_type, data.edge_norm)
    print(out)
    F.nll_loss(out[data.train_idx], data.train_y).backward()
    print(data.train_idx)
    print(out[data.train_idx])
    optimizer.step()